In the following notebook we are going to explain the different steps we have followed to determine which information and which optimisation algorithm we have chosen to make our model.\
The first step consists of importing all the libraries necessary for the development of the model. Among them, there are the three algorithms that we have chosen to carry out the different tests: Decision Tree Classifier, Gradient Boosting Classifier, Random Forest Classifier and Logistic Regression. The reason we have chosen these 4 is because we need a classifier, as our output is going to be a 1, 2 or X. 

In [123]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import os

Next, the database must be fully loaded.

In [124]:
db_path = os.path.join(os.getcwd(), 'laliga.sqlite')
laliga_con = sqlite3.connect(str(db_path))
cursor = laliga_con.cursor()

cursor.execute('SELECT * FROM Matches')
data = cursor.fetchall()

columns = cursor.execute('PRAGMA table_info(Matches)').fetchall()
columns = [col[1] for col in columns]

df = pd.DataFrame(data, columns=columns)

Once we have the database loaded we will make several changes. The first is to discard all NAs in the score column:

In [125]:
df = df[df['score'].notna()]

Once we no longer have NAs in the column we can separate the result into goals scored by the home team (goals_home) and goals scored by the away team (away_home). With these two columns we can calculate new data that can be useful as inputs to the model.

In [126]:
df[['goals_home', 'goals_away']] = df['score'].str.split(':', expand=True).astype(int)

Finally, using the goals scored by each team we can determine which team has won. This new data will be stored in the result column. The 1 value will indicate that the home team has won, the 2 value will indicate that the away team has won and finally the 0 value will indicate that it has been a draw.

In [127]:
results = [
    (df['goals_home'] > df['goals_away']),
    (df['goals_home'] == df['goals_away']),
    (df['goals_home'] < df['goals_away'])
]

df['result'] = np.select(results, [1, 0, 2]) 

Once this process is finished, we can delete the score column, as it does not provide us with any more information.

In [128]:
df = df.drop(['score'], axis = 1)

After all these modifications, the database looks like this:

In [129]:
df.head()

,season,division,matchday,date,time,home_team,away_team,goals_home,goals_away,result
0,1928-1929,1,1,2/10/29,None,Arenas Club,Athletic Madrid,2,3,2
1,1928-1929,1,1,2/10/29,None,Espanyol,Real Unión,3,2,1
2,1928-1929,1,1,2/10/29,None,Real Madrid,Catalunya,5,0,1
3,1928-1929,1,1,2/10/29,None,Donostia,Athletic,1,1,0
4,1928-1929,1,1,2/12/29,None,Racing,Barcelona,0,2,2


Our database contains all results from the 1928-1929 season until the 2020-2021 season. As football has evolved so much, data from older seasons will not provide us with relevant information. Therefore, we only want to select data from the seasons 1980 onwards.\
To do this, we will first convert the date column to datetime format with the convert_todate() function.

In [130]:
def convert_todate(obs):
    split_date = obs.split('/')

    if len(split_date[0]) == 1:
        split_date[0] = '0'+split_date[0]
    if len(split_date[1]) == 1:
        split_date[1] = '0'+split_date[1]
    if int(split_date[2]) > 24:
        split_date[2] = '19'+split_date[2]
    else:
        split_date[2] = '20'+split_date[2]

    return '/'.join(split_date)

In [131]:
df['date'] = df['date'].apply(lambda x: convert_todate(x))
df['date'] = pd.to_datetime(df['date'])
df.head()

,season,division,matchday,date,time,home_team,away_team,goals_home,goals_away,result
0,1928-1929,1,1,1929-02-10,None,Arenas Club,Athletic Madrid,2,3,2
1,1928-1929,1,1,1929-02-10,None,Espanyol,Real Unión,3,2,1
2,1928-1929,1,1,1929-02-10,None,Real Madrid,Catalunya,5,0,1
3,1928-1929,1,1,1929-02-10,None,Donostia,Athletic,1,1,0
4,1928-1929,1,1,1929-02-12,None,Racing,Barcelona,0,2,2


Another column that may be interesting to convert, in this case to integer, is the season column: 

In [132]:
df['season'] = df['season'].str.split('-').str[0].astype(int)

Now, we define a mask that we will use to select only the data of the seasons we want and we will apply it to divide the data into two subsets called train and test.

In [133]:
mask = (df['date'] > '1980-01-01') & (df['date'] <= '2010-01-01')
train = df[mask].copy(deep=True)
mask = df['date'] > '2010-01-01'
test = df[mask].copy(deep=True)

At the moment a lot of our data are strings, for example the name of the teams. In this format they cannot be used by the model and, as we consider them relevant information, we decide to map all the teams and assign a number to them. 

In [134]:
all_teams = {}
for count, team in enumerate(train['home_team'].unique()):
    all_teams[team] = count

train['home_team'] = train['home_team'].map(all_teams)
train['away_team'] = train['away_team'].map(all_teams)

test['home_team'] = test['home_team'].map(all_teams)
test['away_team'] = test['away_team'].map(all_teams)

test.fillna(-1, inplace=True)

At this point we can start testing and begin to build our model. First we will choose which columns to use from the subset train. 

In [135]:
cols = ['home_team', 'away_team', 'division', 'matchday']

As a first approach we chose to use these columns because they are the only ones we can use without starting to process the data. We will use them to train the model and the result column we will use to know the performance of our model.

In [136]:
x_train = train[cols]
y_train = train['result']

x_test = test[cols]
y_test = test['result']

With the data separated we can start creating the model. Let's check the accuracy of each:

Decision Tree Classifier (DTC):

In [137]:
model = DecisionTreeClassifier()
model.fit(x_train, y_train)

prediction = model.predict(x_test)

hits = (y_test == prediction).value_counts()
print(f'Accuracy Tree: {hits[1] / len(y_test)}')

Accuracy Tree: 0.3728761827245905


Gradient Boosting Classifier (GBC):

In [138]:
model2 = GradientBoostingClassifier()
model2.fit(x_train, y_train)

prediction = model2.predict(x_test)

hits = (y_test == prediction).value_counts()
print(f'Accuracy BoostingC: {hits[1] / len(y_test)}')

Accuracy BoostingC: 0.4705463424559976


Random Forest Classifier (RFC):

In [139]:
model3 = RandomForestClassifier()
model3.fit(x_train, y_train)

prediction = model3.predict(x_test)

hits = (y_test == prediction).value_counts()
print(f'Accuracy RandomForest: {hits[1] / len(y_test)}')

Accuracy RandomForest: 0.40970597212330856


Logistic Regression (LR):

In [140]:
model4 = LogisticRegression(max_iter=1000)
model4.fit(x_train, y_train)

prediction = model4.predict(x_test)

hits = (y_test == prediction).value_counts()
print(f'Accuracy Logistic: {hits[1] / len(y_test)}')

Accuracy Logistic: 0.4558958184962865


Let's see a table with the diferents accuracies of the four models:

| Model | Accuracy (%) |
|:----:|:----:|
| DTC | 37.28 |
| GBC | 47.05 |
| RFC | 40.97 |
| LR | 45.56 |

The accuracies of the four models are somewhat poor, this is due to the fact that football is a very complicated sport to predict. However, the models with the best results are the Gradient Boosting Classifier and the Logistic Regression.

Although we know that achieving an accuracy between 40% and 50% is a fairly acceptable result, we want to try to improve the model. To do this, we will try two different things:\
    1. Create a history of goals scored and conceded at home and away for each team.\
    2. Create a winning record for each team.

As we have mapped all the teams in the DB and they are now numbers, we have lost their names, so we must reload the database to retrieve them. Therefore we will repeat all the database changes we made before.

In [141]:
db_path = os.path.join(os.getcwd(), 'laliga.sqlite')
laliga_con = sqlite3.connect(str(db_path))
cursor = laliga_con.cursor()

cursor.execute('SELECT * FROM Matches')
data = cursor.fetchall()

columns = cursor.execute('PRAGMA table_info(Matches)').fetchall()
columns = [col[1] for col in columns]

df = pd.DataFrame(data, columns=columns)

df = df[df['score'].notna()]

df[['goals_home', 'goals_away']] = df['score'].str.split(':', expand=True).astype(int)

results = [
    (df['goals_home'] > df['goals_away']),
    (df['goals_home'] == df['goals_away']),
    (df['goals_home'] < df['goals_away'])
]

df['result'] = np.select(results, [1, 0, 2]) 

df = df.drop(['score'], axis = 1)

df['date'] = df['date'].apply(lambda x: convert_todate(x))
df['date'] = pd.to_datetime(df['date'])

df['season'] = df['season'].str.split('-').str[0].astype(int)

mask = (df['date'] > '1980-01-01') & (df['date'] <= '2010-01-01')
train = df[mask].copy(deep=True)
mask = df['date'] > '2010-01-01'
test = df[mask].copy(deep=True)

We'll map the teams again, but this time we'll run the history of goals and the history of wins functions first.

In [142]:
all_teams = {}
for count, team in enumerate(train['home_team'].unique()):
    all_teams[team] = count

Let's create the function that counts the historical goals:

In [143]:
def count_goals(data):
    sample_dict = {'scored_home':0, 'conceded_home':0, 'scored_away':0, 'conceded_away':0}
    goals_dictionary = { team : sample_dict.copy() for team in data['home_team'].unique()}

    data['scored_home'] = 0
    data['conceded_home'] = 0
    data['scored_away'] = 0
    data['conceded_away'] = 0

    for index, partido in data.iterrows():

        home_team = partido['home_team']
        away_team = partido['away_team']
        
        home_goals = partido['goals_home']
        away_goals = partido['goals_away']

        goals_dictionary[home_team]['scored_home'] += home_goals
        data.at[index, 'scored_home'] = goals_dictionary[home_team]['scored_home']

        goals_dictionary[away_team]['scored_away'] += away_goals
        data.at[index, 'scored_away'] = goals_dictionary[away_team]['scored_away']

        goals_dictionary[home_team]['conceded_home'] += away_goals
        data.at[index, 'conceded_home'] = goals_dictionary[home_team]['conceded_home']

        goals_dictionary[away_team]['conceded_away'] += home_goals
        data.at[index, 'conceded_away'] = goals_dictionary[away_team]['conceded_away']
        
    return data, goals_dictionary

Once created, we must create a function that save this data in the DB:

In [144]:
def add_goals(data, goals_dict):
    data['scored_home'] = -1
    data['conceded_home'] = -1
    data['scored_away'] = -1
    data['conceded_away'] = -1

    for index, partido in data.iterrows():
        home_team = partido['home_team']
        away_team = partido['away_team']

        if home_team  in goals_dict:
            data.at[index, 'scored_home'] = goals_dict[home_team]['scored_home']
            data.at[index, 'conceded_home'] = goals_dict[home_team]['conceded_home']
        
        if away_team in goals_dict:
            data.at[index, 'scored_away'] = goals_dict[away_team]['scored_away']
            data.at[index, 'conceded_away'] = goals_dict[away_team]['conceded_away']

    return data

Now let's create the function that creates the history of victories:

In [145]:
def wins(data):    
    sample_dict = {'win':0, 'tie':0, 'lost':0}
    wins_dictionary = { team : sample_dict.copy() for team in data['home_team'].unique()}

    data['win_home'] = 0
    data['tie_home'] = 0
    data['lost_home'] = 0

    data['win_away'] = 0
    data['tie_away'] = 0
    data['lost_away'] = 0
    
    for index, partido in data.iterrows():

        home_team = partido['home_team']
        away_team = partido['away_team']
        
        if partido['result'] == 1:
            wins_dictionary[home_team]['win'] += 1
            wins_dictionary[away_team]['lost'] += 1

        if partido['result'] == 2:
            wins_dictionary[away_team]['win'] += 1
            wins_dictionary[home_team]['lost'] += 1

        else:
            wins_dictionary[home_team]['tie'] += 1
            wins_dictionary[away_team]['tie'] += 1


        data.at[index, 'win_home'] = wins_dictionary[home_team]['win']
        data.at[index, 'tie_home'] = wins_dictionary[home_team]['tie']
        data.at[index, 'lost_home'] = wins_dictionary[home_team]['lost']

        data.at[index, 'win_away'] = wins_dictionary[away_team]['win']
        data.at[index, 'tie_away'] = wins_dictionary[away_team]['tie']
        data.at[index, 'lost_away'] = wins_dictionary[away_team]['lost']

    return data, wins_dictionary

And, as before, let's create a function that store this new information into the DB:

In [146]:
def add_wins(data, wins_dict):
    data['win'] = 0
    data['tie'] = 0
    data['lost'] = 0

    for index, partido in data.iterrows():
        home_team = partido['home_team']
        away_team = partido['away_team']

        if home_team in wins_dict:
            data.at[index, 'win_home'] = wins_dict[home_team]['win']
            data.at[index, 'tie_home'] = wins_dict[home_team]['tie']
            data.at[index, 'lost_home'] = wins_dict[home_team]['lost']
        
        if away_team in wins_dict:
            data.at[index, 'win_away'] = wins_dict[away_team]['win']
            data.at[index, 'tie_away'] = wins_dict[away_team]['tie']
            data.at[index, 'lost_away'] = wins_dict[away_team]['lost']

    return data

And now, we can write all the new data into the DB:

In [147]:
train, goals_dict = count_goals(train)
test = add_goals(test, goals_dict)

train, wins_dict = wins(train)
test = add_wins(test, wins_dict)

Once the information has been saved, we can re-map all the teams:

In [148]:
train['home_team'] = train['home_team'].map(all_teams)
train['away_team'] = train['away_team'].map(all_teams)

test['home_team'] = test['home_team'].map(all_teams)
test['away_team'] = test['away_team'].map(all_teams)

test.fillna(-1, inplace=True)

Let's take a look of how our train and test DBs looks now:

In [149]:
train.head()

,season,division,matchday,date,time,home_team,away_team,goals_home,goals_away,result,scored_home,conceded_home,scored_away,conceded_away,win_home,tie_home,lost_home,win_away,tie_away,lost_away
10560,1979,1,15,1980-01-06,None,0,15,1,0,1,1,0,0,1,1,1,0,0,1,1
10561,1979,1,15,1980-01-06,None,1,16,1,0,1,1,0,0,1,1,1,0,0,1,1
10562,1979,1,15,1980-01-06,None,2,11,1,1,0,1,1,1,1,0,1,0,0,1,0
10563,1979,1,15,1980-01-06,None,3,10,1,1,0,1,1,1,1,0,1,0,0,1,0
10564,1979,1,15,1980-01-06,None,4,12,0,0,0,0,0,0,0,0,1,0,0,1,0


In [150]:
test.head()

,season,division,matchday,date,time,home_team,away_team,goals_home,goals_away,result,...,conceded_away,win,tie,lost,win_home,tie_home,lost_home,win_away,tie_away,lost_away
21648,2009,1,16,2010-01-02,6:00 PM,3.0,1.0,1,0,1,...,862,0,0,0,498.0,861.0,337.0,390.0,888.0,418.0
21649,2009,1,16,2010-01-02,8:00 PM,17.0,38.0,1,1,0,...,489,0,0,0,639.0,835.0,224.0,258.0,506.0,228.0
21650,2009,1,16,2010-01-02,10:00 PM,6.0,7.0,2,1,1,...,819,0,0,0,506.0,856.0,344.0,460.0,873.0,393.0
21651,2009,1,16,2010-01-03,5:00 PM,16.0,39.0,2,2,0,...,342,0,0,0,407.0,881.0,438.0,158.0,324.0,161.0
21652,2009,1,16,2010-01-03,5:00 PM,43.0,20.0,1,0,1,...,914,0,0,0,155.0,380.0,199.0,379.0,900.0,432.0


First let's look at whether or not the columns that show the goal history add better performance to the models.

In [151]:
cols = ['home_team', 'away_team', 'division', 
'matchday', 'scored_home', 'scored_away', 'conceded_away', 'conceded_home']

x_train = train[cols]
y_train = train['result']

x_test = test[cols]
y_test = test['result']

Decision Tree Classifier (DTC):

In [152]:
model = DecisionTreeClassifier()
model.fit(x_train, y_train)

prediction = model.predict(x_test)

hits = (y_test == prediction).value_counts()
print(f'Accuracy Tree: {hits[1] / len(y_test)}')

Accuracy Tree: 0.37002746973242445


Gradient Boosting Classifier (GBC):

In [153]:
model2 = GradientBoostingClassifier()
model2.fit(x_train, y_train)

prediction = model2.predict(x_test)

hits = (y_test == prediction).value_counts()
print(f'Accuracy BoostingC: {hits[1] / len(y_test)}')

Accuracy BoostingC: 0.4461288025231458


Random Forest Classifier (RFC):

In [154]:
model3 = RandomForestClassifier()
model3.fit(x_train, y_train)

prediction = model3.predict(x_test)

hits = (y_test == prediction).value_counts()
print(f'Accuracy RandomForest: {hits[1] / len(y_test)}')

Accuracy RandomForest: 0.4369722250483264


Logistic Regression (LR):

In [155]:
model4 = LogisticRegression(max_iter=1000)
model4.fit(x_train, y_train)

prediction = model4.predict(x_test)

hits = (y_test == prediction).value_counts()
print(f'Accuracy Logistic: {hits[1] / len(y_test)}')

Accuracy Logistic: 0.47298809644928275


Let's see a table with the diferents accuracies of the four models:

| Model | Accuracy (%) |
|:----:|:----:|
| DTC | 37.00 |
| GBC | 44.61 |
| RFC | 43.69 |
| LR | 47.30 |

If we check the new percentages with the old ones, we can determine that the goal history not only does not have a great effect, but does not improve performance. As an exception we have the logical regression that increases considerably in success and reaches 48%.

Let's take a look at what's going on with the winning record.

In [156]:
cols = ['home_team', 'away_team', 'division', 
'matchday', 'win_home', 'tie_home', 'lost_home',
'win_away', 'tie_away', 'lost_away']

x_train = train[cols]
y_train = train['result']

x_test = test[cols]
y_test = test['result']

Decision Tree Classifier (DTC):

In [157]:
model = DecisionTreeClassifier()
model.fit(x_train, y_train)

prediction = model.predict(x_test)

hits = (y_test == prediction).value_counts()
print(f'Accuracy Tree: {hits[1] / len(y_test)}')

Accuracy Tree: 0.3710448672296266


Gradient Boosting Classifier (GBC):

In [158]:
model2 = GradientBoostingClassifier()
model2.fit(x_train, y_train)

prediction = model2.predict(x_test)

hits = (y_test == prediction).value_counts()
print(f'Accuracy BoostingC: {hits[1] / len(y_test)}')

Accuracy BoostingC: 0.4466375012717469


Random Forest Classifier (RFC):

In [159]:
model3 = RandomForestClassifier()
model3.fit(x_train, y_train)

prediction = model3.predict(x_test)

hits = (y_test == prediction).value_counts()
print(f'Accuracy RandomForest: {hits[1] / len(y_test)}')

Accuracy RandomForest: 0.41977820734560994


Logistic Regression (LR):

In [160]:
model4 = LogisticRegression(max_iter=10000)
model4.fit(x_train, y_train)

prediction = model4.predict(x_test)

hits = (y_test == prediction).value_counts()
print(f'Accuracy Logistic: {hits[1] / len(y_test)}')

Accuracy Logistic: 0.4731915759487232


Let's see a table with the diferents accuracies of the four models:

| Model | Accuracy (%) |
|:----:|:----:|
| DTC | 37.10 |
| GBC | 44.66 |
| RFC | 41.98 |
| LR | 47.32 |

In this case we get very similiar results. The GBC and RFC models get worse another time but the logistic regression improve another time.

Another try that we can perform is to mix the diferents rows that we have obtained with the functions.

In [161]:
cols = ['home_team', 'away_team', 'division', 
'matchday', 'scored_home', 'win_home', 'scored_away',
'win_away']

x_train = train[cols]
y_train = train['result']

x_test = test[cols]
y_test = test['result']

Decision Tree Classifier (DTC):

In [162]:
model = DecisionTreeClassifier()
model.fit(x_train, y_train)

prediction = model.predict(x_test)

hits = (y_test == prediction).value_counts()
print(f'Accuracy Tree: {hits[1] / len(y_test)}')

Accuracy Tree: 0.3760301149659172


Gradient Boosting Classifier (GBC):

In [163]:
model2 = GradientBoostingClassifier()
model2.fit(x_train, y_train)

prediction = model2.predict(x_test)

hits = (y_test == prediction).value_counts()
print(f'Accuracy BoostingC: {hits[1] / len(y_test)}')

Accuracy BoostingC: 0.4692237257096348


Random Forest Classifier (RFC):

In [164]:
model3 = RandomForestClassifier()
model3.fit(x_train, y_train)

prediction = model3.predict(x_test)

hits = (y_test == prediction).value_counts()
print(f'Accuracy RandomForest: {hits[1] / len(y_test)}')

Accuracy RandomForest: 0.44460270627734255


Logistic Regression (LR):

In [165]:
model4 = LogisticRegression(max_iter=1000)
model4.fit(x_train, y_train)

prediction = model4.predict(x_test)

hits = (y_test == prediction).value_counts()
print(f'Accuracy Logistic: {hits[1] / len(y_test)}')

Accuracy Logistic: 0.47685420693865094


Let's see a table with the diferents accuracies of the four models:

| Model | Accuracy (%) |
|:----:|:----:|
| DTC | 37.60 |
| GBC | 46.92 |
| RFC | 44.46 |
| LR | 47.68 |

Finally, the most significant change is thaht we have increased just by a 0.4% the accuracy of the Logistic Regression model. Once finished all this tries it is save to say that the historics of goals and the historics of wins does not increase the efficiency of the models. 

After we have built multiple models to predict a 2020-2021 of LaLiga matchday the conclusion that we get is that a good prediction cannot be done using only these attributes. It is obvious that there are some parameters that are more important than others but none of them it is useful at all. Maybe we would need a parameter that tells us the fitness of the teams or the quality of squads to create a model that is capable to achieve better performances.